In [80]:
import numpy as np
import pandas as pd

path = r'C:\Users\ghoas\OneDrive\Desktop\excel sheets\heart.csv'
data = pd.read_csv(path)
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [81]:
data.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [82]:
sex_dummy = pd.get_dummies(data.Sex, drop_first=True)
chest_paindummy = pd.get_dummies(data.ChestPainType, drop_first=True)
ecg_dummmy = pd.get_dummies(data.RestingECG, drop_first=True)
exe_dummmy = pd.get_dummies(data.ExerciseAngina, drop_first=True)
st_dummmy = pd.get_dummies(data.ST_Slope, drop_first=True)

In [83]:
data = pd.concat([data, sex_dummy,chest_paindummy,ecg_dummmy,exe_dummmy,st_dummmy],axis='columns')

In [84]:
data.isna().any()

Age               False
Sex               False
ChestPainType     False
RestingBP         False
Cholesterol       False
FastingBS         False
RestingECG        False
MaxHR             False
ExerciseAngina    False
Oldpeak           False
ST_Slope          False
HeartDisease      False
M                 False
ATA               False
NAP               False
TA                False
Normal            False
ST                False
Y                 False
Flat              False
Up                False
dtype: bool

In [85]:

data1 = data[data.Cholesterol <= (
    data.Cholesterol.mean()+3*data.Cholesterol.std())]
data1.shape

(915, 21)

In [86]:
data2 = data1[data1.Oldpeak <= (data1.Oldpeak.mean()+3*data1.Oldpeak.std())]
data2.shape

(909, 21)

In [87]:
data3 = data2[data2.RestingBP <= (
    data2.RestingBP.mean()+3*data2.RestingBP.std())]
data3.shape

(902, 21)

In [88]:
data = data3.drop(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina','ST_Slope'],axis='columns')
data.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,M,ATA,NAP,TA,Normal,ST,Y,Flat,Up
0,40,140,289,0,172,0.0,0,True,True,False,False,True,False,False,False,True
1,49,160,180,0,156,1.0,1,False,False,True,False,True,False,False,True,False
2,37,130,283,0,98,0.0,0,True,True,False,False,False,True,False,False,True
3,48,138,214,0,108,1.5,1,False,False,False,False,True,False,True,True,False
4,54,150,195,0,122,0.0,0,True,False,True,False,True,False,False,False,True


In [89]:
X = data.drop(['HeartDisease'], axis='columns')
y=data.HeartDisease

In [90]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42896269,  0.46089071,  0.85238015, ..., -0.82065181,
        -1.00221976,  1.13805334],
       [-0.47545956,  1.5925728 , -0.16132855, ..., -0.82065181,
         0.99778516, -0.87869344],
       [-1.74679706, -0.10495034,  0.79657967, ..., -0.82065181,
        -1.00221976,  1.13805334],
       ...,
       [ 0.37209878, -0.10495034, -0.61703246, ...,  1.21854359,
         0.99778516, -0.87869344],
       [ 0.37209878, -0.10495034,  0.35947592, ..., -0.82065181,
         0.99778516, -0.87869344],
       [-1.64085227,  0.3477225 , -0.20782894, ..., -0.82065181,
        -1.00221976,  1.13805334]])

In [91]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=42)

In [92]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [93]:
#using SVM for prediction


from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(),X,y,cv=5)
scores.mean()

np.float64(0.6895273173726213)

In [94]:
from sklearn.ensemble import BaggingClassifier

bg = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,  # 100 samples
    max_samples=0.8,  # use 80% of samples for each samples
    oob_score=True,
    random_state=0
)
bg.fit(X_train, y_train)
bg.oob_score_

0.871012482662968

In [95]:
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(),X,y,cv=5)
scores.mean()

np.float64(0.7293738489871087)

In [97]:

bg = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,  # 100 samples
    max_samples=0.8,  # use 80% of samples for each samples
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bg, X_train, y_train, cv=5)
scores.mean()

np.float64(0.8502394636015327)